# Constructing a Grid from Points

In many cases, data can be represented as an unstructured series of points, including radar observations or data from climate models when not paired with any connectivity information. UXarray is written around the UGRID conventions, which requires a minimal
set of coordinate and connectivity variables to represent a two-dimensional grid. This notebook demonstrates how grid connectivity can be constructured using point data.


In [1]:
%load_ext autoreload
%autoreload 2

import uxarray as ux
import xarray as xr
import cartopy.crs as ccrs
import geoviews as gv
import geoviews.feature as gf
import warnings

import holoviews as hv

hv.extension("bokeh")

warnings.filterwarnings("ignore")

## Types of Point Data

Understanding the domain 


Different types of point data can be encountered depending on the coverage and structure of the data. The following table categorizes these types, providing examples of common use cases.

| **Domain**        | **Description**                              |
|:---------------------------|:--------------------------------------------------------------------------|
| **Global**              | Data that covers the entire globe (e.g., atmospheric or climate simulations)|
| **Global with Holes**   | Data that spans most of the globe but has gaps or regions without observations (e.g., land-only or ocean-only data). |
| **Regional**            | Data that is limited to a specific area (e.g. local weather forecasts or sateleite observations) |
| **Regional with Holes** | Regional data with missing sections or observations within the area of interest, often due to obstacles or coverage limitations. |


For this notebook, we will be using the coordinates from three testing grids to represent our point data:
* `outCSne30.ug`: Global Cube Sphere Grid
* `oQU480.23010.nc`: Global Ocean Grid
* Subset of `outCSne30.ug`: 9 points centered about (0, 0) 

In [ ]:
uxgrid_global = ux.open_grid("../../test/meshfiles/ugrid/outCSne30/outCSne30.ug")
uxgrid_global_ocean = ux.open_grid("../../test/meshfiles/mpas/QU/oQU480.231010.nc")
uxgrid_global_ocean.normalize_cartesian_coordinates()
uxgrid_regional = uxgrid_global.subset.nearest_neighbor((0.0, 0.0), k=9)

(
    uxgrid_global.plot.face_centers(
        projection=ccrs.Robinson(),
        global_extent=True,
        features=["grid"],
        s=2,
        title="Global Points",
    )
    + uxgrid_global_ocean.plot.face_centers(
        projection=ccrs.Robinson(),
        global_extent=True,
        features=["grid"],
        s=2,
        title="Global Points with Holes",
    )
    + uxgrid_regional.plot.face_centers(
        projection=ccrs.Robinson(),
        global_extent=True,
        features=["grid"],
        s=2,
        title="Regional Points",
    )
).cols(1)

## Preparing Point Data

UXarray's ``Grid.from_points()`` method supports both Spherical (lon and lat) and Cartesian (x, y, z) coordinates. It is important to note that the coordinate arrays must be unique in order to run the following methods.

Below we extract the Cartesian (x, y, z) coordinates which we will use for constructing our grid.


In [ ]:
x_global, y_global, z_global = (
    uxgrid_global.face_x.values,
    uxgrid_global.face_y.values,
    uxgrid_global.face_z.values,
)
points_global = (x_global, y_global, z_global)

In [ ]:
x_global_ocean, y_global_ocean, z_global_ocean = (
    uxgrid_global_ocean.face_x.values,
    uxgrid_global_ocean.face_y.values,
    uxgrid_global_ocean.face_z.values,
)
points_global_ocean = (x_global_ocean, y_global_ocean, z_global_ocean)
boundary_points_global_ocean = uxgrid_global_ocean.boundary_face_indices.values

In [ ]:
x_regional, y_regional, z_regional = (
    uxgrid_regional.face_x.values,
    uxgrid_regional.face_y.values,
    uxgrid_regional.face_z.values,
)
points_regional = (x_regional, y_regional, z_regional)

## Global Data

The following algorithms will returns grids with a full coverage of the surface of a sphere, which makes them suitable for constructing connectivity from global point data.

###  Spherical Delaunay

The `spherical_delaunay` method in the `Grid.from_points()` function is designed to perform Delaunay triangulation on points distributed over a spherical surface. 

#### How It Works

1. **Input Points on the Sphere**:
   - The method accepts input points defined in spherical coordinates (e.g., latitude and longitude) or Cartesian coordinates (x, y, z) that lie on the surface of the sphere. They are internally converted to normalized Cartesian coordinates.

2. **Computing the Convex Hull**:
   - The algorithm computes the **Convex Hull** of the set of Cartesian points. The convex hull is the smallest convex shape that encloses all the points. In three dimensions, the convex hull forms a polyhedron where each face is a triangle.

3. **Extracting Triangles**:
   - Once the convex hull is determined, the triangular faces of the hull are extracted. These triangles represent the Delaunay triangulation on the sphere's surface, ensuring that no point is inside the circumcircle of any triangle, which is a key property of Delaunay triangulations.

4. **Mapping Back to Spherical Coordinates**:
   - The triangles from the convex hull are then mapped back to the original spherical coordinates. This results in a mesh of triangles that covers the spherical surface, suitable for various analyses and visualizations.


In [ ]:
%%time
grid_dt = ux.Grid.from_points(points_global, method="spherical_delaunay")

In [ ]:
grid_dt.plot(
    projection=ccrs.Robinson(),
    line_width=0.5,
    periodic_elements="split",
    title="Spherical Delaunay Triangulation",
)

The resulting grid will always be strictly triangular and cover the entire sphere.

In [ ]:
grid_dt.triangular

In [ ]:
grid_dt.plot.face_degree_distribution()

In [ ]:
grid_dt.plot.face_area_distribution(bins=10)

In [ ]:
grid_dt.global_sphere_coverage

Zooming in, we can observe the Delaunay Triangles in detail. The original point coordinates are now the corners of our faces. This means that any data that was originally mapped to the points will reside on the corner nodes.

In [ ]:
(grid_dt.plot() * uxgrid_global.plot.face_centers(color="red")).opts(
    xlim=(-10, 10),
    ylim=(-5, 5),
    height=400,
    width=400,
    title="Spherical Delaunay Triangles (Zoomed)",
)

### Spherical Voronoi


The `spherical_voronoi` method in the `Grid.from_points()` function is designed to generate a Voronoi tessellation on points distributed over a spherical surface. This method leverages SciPy's Spherical Voronoi functionality internally.

### How It Works

1. **Input Points on the Sphere**:
   - The method accepts input points defined in spherical coordinates (e.g., latitude and longitude) or Cartesian coordinates (x, y, z) that lie on the surface of the sphere. They are internally converted to normalized Cartesian coordinates.

2. **Computing the Spherical Voronoi Diagram**:
   - Using SciPy's `SphericalVoronoi` class, the algorithm computes the Voronoi tessellation on the sphere. This involves determining the regions on the sphere where each region contains all points closer to one generating point than to any other.

3. **Constructing Voronoi Regions**:
   - The Spherical Voronoi algorithm identifies the vertices and edges that define each Voronoi region. Each region corresponds to one input point and consists of a polygon on the sphere's surface.

4. **Mapping to Grid**:
   - The Voronoi regions and coordinates are then encoded in the UGRID conventions for use with a ``Grid``



In [ ]:
%%time
grid_sv = ux.Grid.from_points(points_global, method="spherical_voronoi")

In [ ]:
grid_sv.plot(
    projection=ccrs.Robinson(),
    line_width=0.5,
    periodic_elements="split",
    title="Spherical Voronoi Tesselation",
)

The resulting grid consists of mostly 6-sided faces, with small numbers of faces with 4, 5, 7, and 8 sides.

In [ ]:
grid_sv.plot.face_degree_distribution()

In [ ]:
grid_sv.plot.face_area_distribution(bins=15)

Zooming in, we can observe the Voronoi Regions in detail. The original point coordinates are now the centers of each the faces in the grid. This means that any data that was originally mapped to the points now resides on the faces.

In [ ]:
(grid_sv.plot() * uxgrid_global.plot.face_centers(color="red")).opts(
    xlim=(-10, 10),
    ylim=(-5, 5),
    height=400,
    width=400,
    title="Spherical Voronoi Cells (Zoomed)",
)

In [ ]:
(grid_sv.plot() * uxgrid_global.plot.face_centers(color="red")).opts(
    xlim=(14.5, 18.5),
    ylim=(5.5, 9.0),
    height=400,
    width=400,
    title="Single Spherical Voronoi Cell (Zoomed)",
)

### Limitations of Spherical Methods

In [ ]:
%%time
grid_dt_regional = ux.Grid.from_points(points_regional, method="spherical_delaunay")

In [ ]:
grid_dt_regional.plot.face_area_distribution(
    bins=15, title="Delaunay: Face Area Distributon (Regional)"
)

In [ ]:
grid_sv_regional = ux.Grid.from_points(points_regional, method="spherical_voronoi")

In [ ]:
grid_sv_regional.plot.face_area_distribution(
    bins=15, title="Voronoi: Face Area Distributon (Regional)"
)

## Global Data with Holes

### Spherical Delaunay

In [ ]:
%%time
grid_dt = ux.Grid.from_points(points_global_ocean, method="spherical_delaunay")

In [ ]:
grid_dt.plot(
    projection=ccrs.Robinson(),
    line_width=0.5,
    periodic_elements="exclude",
    title="Spherical Delaunay Triangulation",
)

In [ ]:
(
    grid_dt.plot(
        features=["coastline"],
    )
    * uxgrid_global_ocean.plot.face_centers(color="red")
).opts(
    xlim=(-20, 20),
    ylim=(-10, 10),
    height=400,
    width=400,
    title="Spherical Delaunay Triangles (Zoomed)",
)

In [ ]:
%%time
grid_dt_no_boundary = ux.Grid.from_points(
    points_global_ocean,
    boundary_points=boundary_points_global_ocean,
    method="spherical_delaunay",
)

In [ ]:
grid_dt_no_boundary.plot(
    projection=ccrs.Robinson(),
    line_width=0.5,
    periodic_elements="exclude",
    title="Spherical Delaunay Triangulation (Excluding Boundary Points)",
)

In [ ]:
(
    grid_dt_no_boundary.plot(
        features=["coastline"],
    )
    * uxgrid_global_ocean.plot.face_centers(color="red")
).opts(
    xlim=(-20, 20),
    ylim=(-10, 10),
    height=400,
    width=400,
    title="Spherical Delaunay Triangles (Zoomed)",
)

### Spherical Voronoi

In [ ]:
%%time
grid_sv = ux.Grid.from_points(points_global_ocean, method="spherical_voronoi")

In [ ]:
grid_sv.plot(
    projection=ccrs.Robinson(),
    line_width=0.5,
    periodic_elements="split",
    title="Spherical Voronoi Tesselation",
)

In [ ]:
(
    grid_sv.plot(
        features=["coastline"],
    )
    * uxgrid_global_ocean.plot.face_centers(color="red")
).opts(
    xlim=(-20, 20),
    ylim=(-10, 10),
    height=400,
    width=400,
    title="Spherical Voronoi Regions (Zoomed)",
)

In [ ]:
%%time
grid_sv_no_boundary = ux.Grid.from_points(
    points_global_ocean,
    boundary_points=boundary_points_global_ocean,
    method="spherical_voronoi",
)

In [ ]:
grid_sv_no_boundary.plot(
    projection=ccrs.Robinson(),
    line_width=0.5,
    periodic_elements="split",
    title="Spherical Voronoi Tesselation (Excluding Boundary Points",
)

In [ ]:
(
    grid_sv_no_boundary.plot(
        features=["coastline"],
    )
    * uxgrid_global_ocean.plot.face_centers(color="red")
).opts(
    xlim=(-20, 20),
    ylim=(-10, 10),
    height=400,
    width=400,
    title="Spherical Voronoi Regions (Zoomed)",
)